# Make a catalog of SEDs

This notebook will show you how to generate a Catalog of SED objects and make some plots from the sample.

In [1]:
# Imports
from sedkit import sed, spectrum, synphot, catalog
import astropy.units as q
import numpy as np
import copy
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading BokehJS ...

First we initialize an empty catalog.

In [2]:
# Make an SED catalog
cat = catalog.Catalog()

Now let's make a few SEDs

In [3]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='L3', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('sedkit', 'data/L3_photometry.txt'))
cat.add_SED(s1)

Setting radius to (<Quantity 0.09415542 solRad>, <Quantity 0.01964052 solRad>)
Setting age to (<Quantity 6. Gyr>, <Quantity 4. Gyr>)


In [4]:
# Make 10 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.name += str(i)
    cat.add_SED(s2)

Then the (abridged) aggregated results look like this:

In [5]:
cat.results[['name','distance','distance_unc','Lbol_sun','Lbol_sun_unc','Teff','Teff_unc']]

name,distance,distance_unc,Lbol_sun,Lbol_sun_unc,Teff,Teff_unc
,pc,pc,,,K,K
object,object,object,object,object,object,object
2MASS 261832886,572.28,16.41,-0.317,0.025,15675.0,1650.0
2MASS 2618328860,545.12,29.72,-0.359,0.048,15298.0,1649.0
2MASS 2618328861,555.83,30.89,-0.342,0.049,15448.0,1668.0
2MASS 2618328862,590.14,34.83,-0.29,0.052,15918.0,1726.0
2MASS 2618328863,520.4,27.08,-0.399,0.046,14947.0,1607.0
2MASS 2618328864,577.97,33.4,-0.308,0.05,15753.0,1705.0
2MASS 2618328865,601.29,36.15,-0.274,0.052,16067.0,1744.0
2MASS 2618328866,554.73,30.77,-0.344,0.048,15433.0,1666.0


We can also plot the results by passing the `plot` method two column names.

In [6]:
show(cat.plot('distance', 'Teff'))

Or plot a CMD! You can also pass the difference of two magnitudes as a string like so.

In [7]:
show(cat.plot('2MASS.J-2MASS.Ks', 'M_2MASS.J'))

We can then retrieve the SED for some sources like so:

In [42]:
show(cat.plot_SEDs(['2MASS 261832886', '2MASS 2618328865', '2MASS 2618328863'], app=False))

The catalog can also be filtered by providing a column name and value...

In [43]:
cat.filter('name', "'2MASS 2618328861'").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 2618328861,6.0,4.0,555.83,30.89,1.7990997028572533,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.74134119504054e+33,1.9469581732469965e+32,-0.342,0.049,5.61,0.062,4.55,0.04,0.85,0.04,15448.0,1668.0,5216.0,80.0,None,<sedkit.sed.SED object at 0x128703c50>,24.69,1.389,15.96,1.391,22.53,0.137,13.805,0.15,19.69,0.019,10.96,0.063,17.17,0.005,8.445,0.061,15.22,0.005,6.492,0.061,12.47,0.027,3.742,0.066,11.586,0.03,2.86,0.067,11.055,0.021,2.328,0.064


...or a wildcard search...

In [44]:
cat.filter('name', "*328861").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 2618328861,6.0,4.0,555.83,30.89,1.7990997028572533,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.74134119504054e+33,1.9469581732469965e+32,-0.342,0.049,5.61,0.062,4.55,0.04,0.85,0.04,15448.0,1668.0,5216.0,80.0,None,<sedkit.sed.SED object at 0x128703c50>,24.69,1.389,15.96,1.391,22.53,0.137,13.805,0.15,19.69,0.019,10.96,0.063,17.17,0.005,8.445,0.061,15.22,0.005,6.492,0.061,12.47,0.027,3.742,0.066,11.586,0.03,2.86,0.067,11.055,0.021,2.328,0.064


... or an argument with an operator...

In [45]:
cat.filter('logg', '>=4.5').results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 261832886,6.0,4.0,572.28,16.41,1.7474,0.0501,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.845937720647223e+33,1.0820205056393471e+32,-0.317,0.025,5.547,0.034,4.54,0.04,0.86,0.03,15675.0,1650.0,5259.0,39.0,None,<sedkit.sed.SED object at 0x128204cf8>,24.69,1.389,15.9,1.389,22.53,0.137,13.74,0.14,19.69,0.019,10.9,0.036,17.17,0.005,8.38,0.032,15.22,0.005,6.43,0.032,12.47,0.027,3.68,0.041,11.586,0.03,2.797,0.043,11.055,0.021,2.266,0.038
2MASS 2618328860,6.0,4.0,545.12,29.72,1.8344612016193196,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.6748817096515836e+33,1.8375420459200495e+32,-0.359,0.048,5.653,0.061,4.55,0.04,0.85,0.04,15298.0,1649.0,5190.0,82.0,None,<sedkit.sed.SED object at 0x128648dd8>,24.69,1.389,16.0,1.391,22.53,0.137,13.85,0.149,19.69,0.019,11.01,0.062,17.17,0.005,8.49,0.059,15.22,0.005,6.54,0.059,12.47,0.027,3.79,0.065,11.586,0.03,2.906,0.066,11.055,0.021,2.377,0.063
2MASS 2618328861,6.0,4.0,555.83,30.89,1.7990997028572533,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.74134119504054e+33,1.9469581732469965e+32,-0.342,0.049,5.61,0.062,4.55,0.04,0.85,0.04,15448.0,1668.0,5216.0,80.0,None,<sedkit.sed.SED object at 0x128703c50>,24.69,1.389,15.96,1.391,22.53,0.137,13.805,0.15,19.69,0.019,10.96,0.063,17.17,0.005,8.445,0.061,15.22,0.005,6.492,0.061,12.47,0.027,3.742,0.066,11.586,0.03,2.86,0.067,11.055,0.021,2.328,0.064
2MASS 2618328862,6.0,4.0,590.14,34.83,1.6945243506164922,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.9629535016624336e+33,2.3292485912816197e+32,-0.29,0.052,5.48,0.065,4.53,0.05,0.86,0.03,15918.0,1726.0,5297.0,77.0,None,<sedkit.sed.SED object at 0x128734f98>,24.69,1.389,15.83,1.391,22.53,0.137,13.68,0.151,19.69,0.019,10.83,0.067,17.17,0.005,8.32,0.064,15.22,0.005,6.367,0.064,12.47,0.027,3.617,0.07,11.586,0.03,2.734,0.071,11.055,0.021,2.203,0.067
2MASS 2618328863,6.0,4.0,520.4,27.08,1.9215900422078405,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.526421530663735e+33,1.5993411977221359e+32,-0.399,0.046,5.753,0.058,4.56,0.04,0.84,0.04,14947.0,1607.0,5152.0,113.0,None,<sedkit.sed.SED object at 0x1286dac50>,24.69,1.389,16.11,1.39,22.53,0.137,13.95,0.148,19.69,0.019,11.11,0.06,17.17,0.005,8.59,0.057,15.22,0.005,6.64,0.057,12.47,0.027,3.89,0.063,11.586,0.03,3.008,0.064,11.055,0.021,2.477,0.06
2MASS 2618328864,6.0,4.0,577.97,33.4,1.73020167

... or even a tuple of arguments...

In [46]:
cat.filter('M_SDSS.r', ('>10.9','<11')).results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,SpT,SpT_fit,membership,reddening,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,Teff_evo,Teff_evo_unc,Teff_bb,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc
,Gyr,Gyr,pc,pc,mas,mas,solRad,solRad,,,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,K,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16,float16
2MASS 2618328861,6.0,4.0,555.83,30.89,1.7990997028572533,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.74134119504054e+33,1.9469581732469965e+32,-0.342,0.049,5.61,0.062,4.55,0.04,0.85,0.04,15448.0,1668.0,5216.0,80.0,None,<sedkit.sed.SED object at 0x128703c50>,24.69,1.389,15.96,1.391,22.53,0.137,13.805,0.15,19.69,0.019,10.96,0.063,17.17,0.005,8.445,0.061,15.22,0.005,6.492,0.061,12.47,0.027,3.742,0.066,11.586,0.03,2.86,0.067,11.055,0.021,2.328,0.064
2MASS 2618328866,6.0,4.0,554.73,30.77,1.8026722275761933,0.1,0.09415541971634411,0.019640519289763958,73.0,0.5,None,None,None,0,4.710738027543608e-11,5.709534127198549e-13,14.335,0.013,1.7344557087216903e+33,1.9356001731828567e+32,-0.344,0.048,5.615,0.062,4.55,0.04,0.85,0.04,15433.0,1666.0,5213.0,79.0,None,<sedkit.sed.SED object at 0x1284fa898>,24.69,1.389,15.97,1.391,22.53,0.137,13.81,0.15,19.69,0.019,10.97,0.063,17.17,0.005,8.45,0.06,15.22,0.005,6.5,0.06,12.47,0.027,3.75,0.066,11.586,0.03,2.867,0.067,11.055,0.021,2.336,0.064
